# PYTORCH_forecasting를 사용한 전력사용량 예측


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/inpu'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install torch -f https://download.pytorch.org/whl/torch_stable.html.

Looking in links: https://download.pytorch.org/whl/torch_stable.html.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pytorch-forecasting

     |████████████████████████████████| 112 kB 834 kB/s eta 0:00:01


In [4]:
pip install torch -f https://download.pytorch.org/whl/torch_stable.html.

Looking in links: https://download.pytorch.org/whl/torch_stable.html.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt

import datetime

import torch
import torch.nn as nn
from torch.autograd import Variable 

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

참고 : https://coding-yoon.tistory.com/131

In [6]:
import copy
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [7]:
train_df=pd.read_csv('../input/energy-dacon/train.csv',encoding = 'cp949')
test_df=pd.read_csv('../input/energy-dacon/test.csv',encoding = 'cp949')
submission_df=pd.read_csv('../input/energy-dacon/sample_submission.csv',encoding = 'cp949')

In [8]:
train_df.dtypes

num             int64
date_time      object
전력사용량(kWh)    float64
기온(°C)        float64
풍속(m/s)       float64
습도(%)         float64
강수량(mm)       float64
일조(hr)        float64
비전기냉방설비운영     float64
태양광보유         float64
dtype: object

In [9]:
train_df.isna().sum()

num           0
date_time     0
전력사용량(kWh)    0
기온(°C)        0
풍속(m/s)       0
습도(%)         0
강수량(mm)       0
일조(hr)        0
비전기냉방설비운영     0
태양광보유         0
dtype: int64

In [10]:
test_df.isna().sum()

num                0
date_time          0
기온(°C)          6720
풍속(m/s)         6720
습도(%)           6720
강수량(mm, 6시간)    8400
일조(hr, 3시간)     6720
비전기냉방설비운영       7784
태양광보유           8456
dtype: int64

In [11]:
temp_df=train_df[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train_df), len(train_df)//60):
    count +=1
    ice[count]=train_df.loc[i,'비전기냉방설비운영']
    hot[count]=train_df.loc[i,'태양광보유']
    
    

In [12]:
test_EDA=copy.deepcopy(test_df)

In [13]:
for i in range(len(test_df)):
    test_EDA.loc[i, '비전기냉방설비운영']=ice[test_df['num'][i]]
    test_EDA.loc[i, '태양광보유']=hot[test_df['num'][i]]

In [14]:
test_EDA=test_EDA.interpolate(method='values')

In [15]:
def change_date(date_one_time):
    dateformat = "%Y-%m-%d %H"
    convert_one_time = datetime.datetime.strptime(date_one_time, dateformat)
    return convert_one_time

def insert_dateindex(df):
    date_time=df['date_time']
    convert_time=date_time.map(change_date)
    date_index=pd.DatetimeIndex(convert_time)
    df.index=date_index
    return df

def change_date_df(df):
    date_time=df['date_time']
    convert_time=date_time.map(change_date)
    df['date_time']=convert_time
    return df
    

In [16]:
test_EDA

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0


In [17]:
train_EDA=copy.deepcopy(train_df)
train_EDA=change_date_df(train_EDA)


In [18]:
num1_train=train_EDA[train_EDA['num']==1]

In [19]:
train_EDA['time']=0

In [20]:
time_dict=num1_train.to_dict()['date_time']

In [21]:
inv_map = {v: k for k, v in time_dict.items()}

In [22]:
train_EDA['time']=train_EDA['date_time'].map(inv_map)

In [23]:
train_EDA['ice']=str("N")
train_EDA['sun']=str("N")

In [24]:
train_EDA['ice']=train_EDA['비전기냉방설비운영'].apply(lambda x:"Yes" if(x==1.0) else "No"  )
train_EDA['sun']=train_EDA['태양광보유'].map({0.0:"N", 1.0:"Y"})

In [25]:
train_EDA[['ice','sun']]=train_EDA[['ice','sun']].astype(str)

In [26]:
train_EDA.dtypes

num                    int64
date_time     datetime64[ns]
전력사용량(kWh)           float64
기온(°C)               float64
풍속(m/s)              float64
습도(%)                float64
강수량(mm)              float64
일조(hr)               float64
비전기냉방설비운영            float64
태양광보유                float64
time                   int64
ice                   object
sun                   object
dtype: object

In [27]:
train_EDA.dtypes

num                    int64
date_time     datetime64[ns]
전력사용량(kWh)           float64
기온(°C)               float64
풍속(m/s)              float64
습도(%)                float64
강수량(mm)              float64
일조(hr)               float64
비전기냉방설비운영            float64
태양광보유                float64
time                   int64
ice                   object
sun                   object
dtype: object

In [28]:
drop_list=['date_time','비전기냉방설비운영','태양광보유','일조(hr)','강수량(mm)','풍속(m/s)','ice','sun','습도(%)']

new_train=train_EDA.drop(drop_list,axis=1)
#x_drop_list=['전력사용량(kWh)']

In [29]:
new_train['num'] = new_train['num'].apply(str)

In [30]:
new_train['time']+=1

In [31]:
new_train

,num,전력사용량(kWh),기온(°C),time
0,1,8179.056,17.6,1
1,1,8135.640,17.7,2
2,1,8107.128,17.5,3
3,1,8048.808,17.1,4
4,1,8043.624,17.0,5
...,...,...,...,...
122395,60,4114.368,27.8,2036
122396,60,3975.696,27.3,2037
122397,60,3572.208,27.3,2038
122398,60,3299.184,27.1,2039


In [32]:
new_train.dtypes

num            object
전력사용량(kWh)    float64
기온(°C)        float64
time            int64
dtype: object

In [33]:
test_EDA[test_EDA['num']==1]

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.0,0.0,0.0
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.0,0.0,0.0
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.0,0.0,0.0
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.0,0.0,0.0
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
163,1,2020-08-31 19,28.233333,2.800000,72.666667,0.0,0.8,0.0,0.0
164,1,2020-08-31 20,27.666667,2.600000,73.333333,0.0,0.4,0.0,0.0
165,1,2020-08-31 21,27.100000,2.400000,74.000000,0.0,0.0,0.0,0.0
166,1,2020-08-31 22,26.966667,1.900000,78.333333,0.0,0.0,0.0,0.0


In [34]:
# group_ids=['기온(°C)','num']
# time_varying_unknown_reals=['기온(°C)',"전력사용량(kWh)"]
# max_prediction_length = 510
# max_encoder_length = 1530
# training_cutoff= df_temp["time"].max() - max_prediction_length
# training=TimeSeriesDataSet (data=df_temp[lambda x: x.time <= training_cutoff] , 
#                             time_idx ="time" , 
#                             target="전력사용량(kWh)" , 
#                             group_ids =group_ids,
#                             allow_missing_timesteps = True,
#                             min_encoder_length=max_encoder_length//2,
#                             max_encoder_length=max_encoder_length,
#                             min_prediction_length=1,
#                             max_prediction_length=max_prediction_length,
#                             static_categoricals=['num'],
#                             time_varying_known_reals=['time'],
#                             time_varying_unknown_reals=time_varying_unknown_reals,
#                             add_relative_time_idx=True,
#                             add_target_scales=True,
#                             add_encoder_length=True,
#                             )

In [35]:
#     validation = TimeSeriesDataSet.from_dataset(training,df_temp, predict=True, stop_randomization=True)
#     batch_size = 32
#     train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
#     val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)

In [36]:
# new_train.columns.tolist()

In [37]:

# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-7, patience=10, verbose=False, mode="min")
# lr_logger = LearningRateMonitor()  
# logger = TensorBoardLogger("lightning_logs") 

# trainer = pl.Trainer(
#     max_epochs=30,
#     gpus=1,
#     weights_summary="top",
#     gradient_clip_val=0.1,
#     limit_train_batches=30,  
#     callbacks=[lr_logger, early_stop_callback],
#     logger=logger,
# )

In [38]:
def num_by_tft(df_origin,num):
    #df=df_origin[(df_origin['num']==str(num))|(df_origin['num']==str(num+1))]
    #df=df.drop('num',axis=1)
    #df=df_origin[(df_origin['num']==str(num))|(df_origin['num']==str(num+1))|(df_origin['num']==str(num+2))|(df_origin['num']==str(num+3))|(df_origin['num']==str(num+4))|(df_origin['num']==str(num+5))]
    df=df_origin
    group_ids=['기온(°C)','num']
    time_varying_unknown_reals=['기온(°C)',"전력사용량(kWh)"]
    max_prediction_length = 168
    #max_encoder_length = 1530
    max_encoder_length = 30
    training_cutoff= df_temp["time"].max() - max_prediction_length
    training=TimeSeriesDataSet (data=df[lambda x: x.time <= training_cutoff] , 
                                time_idx ="time" , 
                                target="전력사용량(kWh)" , 
                                group_ids =group_ids,
                                allow_missing_timesteps = True,
                                min_encoder_length=max_encoder_length//2,
                                max_encoder_length=max_encoder_length,
                                min_prediction_length=1,
                                max_prediction_length=max_prediction_length,
                                static_categoricals=['num'],
                                time_varying_known_reals=['time'],
                                time_varying_unknown_reals=time_varying_unknown_reals,
                                add_relative_time_idx=True,
                                add_target_scales=True,
                                add_encoder_length=True,
                                )
    validation = TimeSeriesDataSet.from_dataset(training,df, predict=True, stop_randomization=True)
    batch_size = 32
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)
    
    tft = TemporalFusionTransformer.from_dataset(
        training,
        # not meaningful for finding the learning rate but otherwise very important
        learning_rate=0.03,
        hidden_size=16,  # most important hyperparameter apart from learning rate
        # number of attention heads. Set to up to 4 for large datasets
        attention_head_size=1,
        dropout=0.1,  # between 0.1 and 0.3 are good values
        hidden_continuous_size=8,  # set to <= hidden_size
        output_size=1,  # 7 quantiles by default
        loss=pytorch_forecasting.metrics.RMSE(),
        # reduce learning rate if no improvement in validation loss after x epochs
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")
    
    
    trainer.fit(
        tft, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
    )
    return tft,val_dataloader




    
    
    

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # device
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [40]:
import pytorch_forecasting

In [41]:
from pytorch_forecasting.data import (
     TimeSeriesDataSet,
     GroupNormalizer
 )

In [42]:
# max_prediction_length = 510
# max_encoder_length = 1530
# time_varying_unknown_reals=['기온(°C)',"전력사용량(kWh)"]
# training=TimeSeriesDataSet (data=new_train , 
#                             time_idx ="time" , 
#                             target="전력사용량(kWh)" , 
#                             group_ids =['num'],
#                             allow_missing_timesteps = True,
#                             min_encoder_length=max_encoder_length,
#                             max_encoder_length=max_encoder_length,
#                             min_prediction_length=1,
#                             max_prediction_length=max_prediction_length,
#                             #static_categoricals=['num'],
#                             time_varying_known_reals=['time'],
#                             time_varying_unknown_reals=time_varying_unknown_reals,
#                             add_relative_time_idx=True,
#                             add_target_scales=True,
#                             add_encoder_length=True,
#                             )

                           


In [43]:
# validation = TimeSeriesDataSet.from_dataset(training, new_train, predict=True, stop_randomization=True)
# batch_size = 40
# train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
# val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)


In [44]:
import pytorch_lightning as pl

In [45]:
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning.loggers import TensorBoardLogger

In [46]:

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-7, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  
logger = TensorBoardLogger("lightning_logs") 

trainer = pl.Trainer(
    max_epochs=30,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

In [47]:

# tft = TemporalFusionTransformer.from_dataset(
#     training,
#     # not meaningful for finding the learning rate but otherwise very important
#     learning_rate=0.03,
#     hidden_size=16,  # most important hyperparameter apart from learning rate
#     # number of attention heads. Set to up to 4 for large datasets
#     attention_head_size=1,
#     dropout=0.1,  # between 0.1 and 0.3 are good values
#     hidden_continuous_size=8,  # set to <= hidden_size
#     output_size=1,  # 7 quantiles by default
#     loss=pytorch_forecasting.metrics.RMSE(),
#     # reduce learning rate if no improvement in validation loss after x epochs
#     reduce_on_plateau_patience=4,
# )

In [48]:
# print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [49]:
# res = trainer.tuner.lr_find(
#     tft, train_dataloader=train_dataloader, 
#     val_dataloaders=val_dataloader, early_stop_threshold=1000.0, max_lr=3, min_lr=1e-6
# )


In [50]:
# print(f"suggested learning rate: {res.suggestion()}")
# fig = res.plot(show=True, suggest=True)
# fig.show()

In [51]:
# trainer.fit(
#     tft, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
# )


In [52]:
torch.cuda.empty_cache()

In [53]:
new_train["time"].max()

2040

In [54]:
#data=new_train[lambda x: x.time <= training_cutoff]
group_ids=['기온(°C)','num']
time_varying_unknown_reals=['기온(°C)',"전력사용량(kWh)"]
max_prediction_length = 180
#max_encoder_length = 1530
max_encoder_length = 2040
training_cutoff= new_train["time"].max() - max_prediction_length
training=TimeSeriesDataSet (data=new_train[lambda x: x.time <= training_cutoff], 
                            time_idx ="time" , 
                            target="전력사용량(kWh)" , 
                            group_ids =group_ids,
                            allow_missing_timesteps = True,
                            min_encoder_length=max_encoder_length//2,
                            max_encoder_length=max_encoder_length,
                            min_prediction_length=1,
                            max_prediction_length=max_prediction_length,
                            static_categoricals=['num'],
                            time_varying_known_reals=['time'],
                            time_varying_unknown_reals=time_varying_unknown_reals,
                            add_relative_time_idx=True,
                            add_target_scales=True,
                            #add_encoder_length=True,
                            )
validation = TimeSeriesDataSet.from_dataset(training,new_train, predict=True, stop_randomization=True)
batch_size = 32
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*10, num_workers=0)

tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=1,  # 7 quantiles by default
    loss=pytorch_forecasting.metrics.RMSE(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


trainer.fit(
    tft, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
)


Number of parameters in network: 20.5k


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [ ]:
# actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
# predictions = tft.predict(val_dataloader)
# (actuals - predictions).abs().mean()

In [ ]:
# raw_predictions, x = tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
# for idx in range(2):  # plot 10 examples
#     tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True)

In [1]:
predictions, x = tft.predict(val_dataloader,return_x=True)
predictions_vs_actuals = tft.calculate_prediction_actual_by_variable(x, predictions)
tft.plot_prediction_actual_by_variable(predictions_vs_actuals);

NameError: name 'tft' is not defined

In [ ]:
# tft.transform_output(predictions)

In [ ]:
val_dataloader

In [ ]:
predictions1, x1 = tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
predictions1

In [ ]:
df_pred.mean(axis=1).plot()

In [56]:
drop_list=['date_time','비전기냉방설비운영','태양광보유','일조(hr, 3시간)','강수량(mm, 6시간)','풍속(m/s)','ice','sun']


test_EDA

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0


In [57]:
test_EDA['time']=0
num1_test=test_EDA[test_EDA['num']==1]
num1_test_index=num1_test.reset_index()

In [58]:
num1_test_index

,index,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time
0,0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.0,0.0,0.0,0
1,1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.0,0.0,0.0,0
2,2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.0,0.0,0.0,0
3,3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.0,0.0,0.0,0
4,4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
163,163,1,2020-08-31 19,28.233333,2.800000,72.666667,0.0,0.8,0.0,0.0,0
164,164,1,2020-08-31 20,27.666667,2.600000,73.333333,0.0,0.4,0.0,0.0,0
165,165,1,2020-08-31 21,27.100000,2.400000,74.000000,0.0,0.0,0.0,0.0,0
166,166,1,2020-08-31 22,26.966667,1.900000,78.333333,0.0,0.0,0.0,0.0,0


In [67]:
drop_list=['date_time','비전기냉방설비운영','태양광보유','일조(hr, 3시간)','강수량(mm, 6시간)','풍속(m/s)','습도(%)']

In [59]:
num1_test_index['index']=num1_test_index['index']+2041

In [60]:
time_dict_test=num1_test_index.set_index('index').to_dict()['date_time']

In [65]:

inv_map_test = {v: k for k, v in time_dict_test.items()}

In [62]:
inv_map_test

{'2020-08-25 00': 2041,
 '2020-08-25 01': 2042,
 '2020-08-25 02': 2043,
 '2020-08-25 03': 2044,
 '2020-08-25 04': 2045,
 '2020-08-25 05': 2046,
 '2020-08-25 06': 2047,
 '2020-08-25 07': 2048,
 '2020-08-25 08': 2049,
 '2020-08-25 09': 2050,
 '2020-08-25 10': 2051,
 '2020-08-25 11': 2052,
 '2020-08-25 12': 2053,
 '2020-08-25 13': 2054,
 '2020-08-25 14': 2055,
 '2020-08-25 15': 2056,
 '2020-08-25 16': 2057,
 '2020-08-25 17': 2058,
 '2020-08-25 18': 2059,
 '2020-08-25 19': 2060,
 '2020-08-25 20': 2061,
 '2020-08-25 21': 2062,
 '2020-08-25 22': 2063,
 '2020-08-25 23': 2064,
 '2020-08-26 00': 2065,
 '2020-08-26 01': 2066,
 '2020-08-26 02': 2067,
 '2020-08-26 03': 2068,
 '2020-08-26 04': 2069,
 '2020-08-26 05': 2070,
 '2020-08-26 06': 2071,
 '2020-08-26 07': 2072,
 '2020-08-26 08': 2073,
 '2020-08-26 09': 2074,
 '2020-08-26 10': 2075,
 '2020-08-26 11': 2076,
 '2020-08-26 12': 2077,
 '2020-08-26 13': 2078,
 '2020-08-26 14': 2079,
 '2020-08-26 15': 2080,
 '2020-08-26 16': 2081,
 '2020-08-26 17'

In [ ]:
test_EDA

In [68]:

#num1_test_index['index']+=2040

test_EDA['time']=test_EDA['date_time'].map(inv_map_test)

new_test=test_EDA.drop(drop_list,axis=1)

In [69]:
new_test['전력사용량(kWh)']=0

In [ ]:
new_test['time'].max()-new_train['time'].max()

In [70]:
new_test['num']=new_test['num'].astype(str)

In [ ]:
# train_newtime['time'].max()

In [ ]:

# drop_list=['num','date_time']
# test_temp=num1_test.drop(drop_list,axis=1)
# test_newindex=test_temp.reset_index()
# test_newtime=test_newindex.rename(columns={'index':'time'})
# test_newtime['time']=test_newtime['time']+2040

In [ ]:
# test_newtime['time']

In [ ]:
submission_df

In [ ]:
# raw_col=train_newtime.columns
# raw_col=train_newtime.drop('전력사용량(kWh)',axis=1).columns
# test_newtime.columns
# test_newtime.columns=raw_col

In [75]:
new_data=pd.concat([new_train,new_test],ignore_index=True)

In [ ]:
new_train

In [73]:
new_data

,num,전력사용량(kWh),기온(°C),time
0,1,8179.056,17.600000,1
1,1,8135.640,17.700000,2
2,1,8107.128,17.500000,3
3,1,8048.808,17.100000,4
4,1,8043.624,17.000000,5
...,...,...,...,...
132475,60,0.000,28.633333,2204
132476,60,0.000,28.266667,2205
132477,60,0.000,27.900000,2206
132478,60,0.000,27.900000,2207


In [ ]:
new_raw_predictions, new_x = tft.predict(new_data,mode="raw",return_x=True)


In [ ]:
new_raw_predictions.shape

In [ ]:
tft.plot_prediction(new_x, new_raw_predictions, idx=1, show_future_observed=False);

In [ ]:
new_data=pd.concat([new_train,new_test]).reset_index(drop=True)

In [ ]:
new_concat_predictions, concat_x = tft.predict(new_data, mode="raw",return_x=True)


In [ ]:
tft.plot_prediction(concat_x, new_concat_predictions, idx=0, show_future_observed=False)

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
val_predictions = best_tft.predict(val_dataloader)

In [ ]:
new_concat_predictions

In [ ]:
pred_value=new_concat_predictions['prediction']

In [ ]:
pred_value

In [ ]:
pred_numpy=pred_value.numpy()

In [ ]:
pred_numpy.shape

In [ ]:
pred_df=pd.DataFrame(pred_numpy[2])

In [ ]:
pred_df

In [ ]:
# submission_1=copy.deepcopy(submission_df)

In [ ]:
# submission_1['answer']=data_predict_inverse

In [ ]:
# submission_1.to_csv('lstm_submission1.csv', index=False)

In [ ]:
# train_preds = np.zeros((len(train_X)))
# test_preds = np.zeros((len(test_X)))

# x_test_cuda = torch.tensor(test_X, dtype=torch.long).cuda()
# test = torch.utils.data.TensorDataset(x_test_cuda)
# test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
